<a href="https://colab.research.google.com/github/senmos26/BOOKSTORE-MERN-STACK/blob/main/TP_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder \
    .appName("FIFA Spark SQL Lab") \
    .getOrCreate()


In [2]:
df = spark.read.csv(
    "fifaworldcup.csv",
    header=True,
    inferSchema=True
)


In [3]:
df.createOrReplaceTempView("matches")


In [5]:
spark.sql("""
SELECT COUNT(*) AS total_matchs
FROM matches
""").show()


+------------+
|total_matchs|
+------------+
|       28367|
+------------+



In [7]:
spark.sql("""
SELECT
    MIN(YEAR(date)) AS premiere_annee,
    MAX(YEAR(date)) AS derniere_annee
FROM matches
""").show()

+--------------+--------------+
|premiere_annee|derniere_annee|
+--------------+--------------+
|          1872|          2005|
+--------------+--------------+



In [8]:
spark.sql("""
SELECT tournament, COUNT(*) AS nb_matchs
FROM matches
GROUP BY tournament
ORDER BY nb_matchs DESC
LIMIT 10
""").show()


+--------------------+---------+
|          tournament|nb_matchs|
+--------------------+---------+
|            Friendly|    12017|
|FIFA World Cup qu...|     4512|
|UEFA Euro qualifi...|     1510|
|African Cup of Na...|     1134|
|        Copa América|      677|
|      FIFA World Cup|      644|
|  Merdeka Tournament|      579|
|British Championship|      505|
|AFC Asian Cup qua...|      459|
|African Cup of Na...|      417|
+--------------------+---------+



In [9]:
spark.sql("""
SELECT COUNT(*) AS matchs_neutres
FROM matches
WHERE neutral = true
""").show()


+--------------+
|matchs_neutres|
+--------------+
|          6420|
+--------------+



In [10]:
spark.sql("""
SELECT country, COUNT(*) AS nb_matchs
FROM matches
GROUP BY country
ORDER BY nb_matchs DESC
LIMIT 10
""").show()


+-------------+---------+
|      country|nb_matchs|
+-------------+---------+
|United States|      642|
|     Malaysia|      606|
|       France|      502|
|      England|      476|
|       Sweden|      463|
|        Italy|      395|
|       Brazil|      388|
|      Germany|      387|
|        Spain|      380|
|    Argentina|      372|
+-------------+---------+



In [11]:
spark.sql("""
SELECT COUNT(*) AS matchs_nuls
FROM matches
WHERE home_score = away_score
""").show()


+-----------+
|matchs_nuls|
+-----------+
|       6484|
+-----------+



In [12]:
spark.sql("""
SELECT team, COUNT(*) AS total_matchs
FROM (
    SELECT home_team AS team FROM matches
    UNION ALL
    SELECT away_team AS team FROM matches
)
GROUP BY team
ORDER BY total_matchs DESC
""").show()


+-----------+------------+
|       team|total_matchs|
+-----------+------------+
|    England|         842|
|     Sweden|         834|
|    Hungary|         796|
|  Argentina|         795|
|     Brazil|         790|
|    Germany|         757|
|    Uruguay|         717|
|     Norway|         665|
|   Scotland|         664|
|South Korea|         662|
|    Austria|         652|
|     France|         651|
|Switzerland|         648|
|    Denmark|         647|
|     Poland|         641|
|     Mexico|         633|
|      Italy|         621|
|    Belgium|         615|
|Netherlands|         612|
|    Finland|         599|
+-----------+------------+
only showing top 20 rows


In [13]:
spark.sql("""
SELECT team, SUM(goals) AS buts_marques
FROM (
    SELECT home_team AS team, home_score AS goals FROM matches
    UNION ALL
    SELECT away_team AS team, away_score AS goals FROM matches
)
GROUP BY team
ORDER BY buts_marques DESC
LIMIT 10
""").show()


+-----------+------------+
|       team|buts_marques|
+-----------+------------+
|    England|        1871|
|     Brazil|        1745|
|     Sweden|        1743|
|    Hungary|        1716|
|    Germany|        1661|
|  Argentina|        1482|
|Netherlands|        1271|
|South Korea|        1211|
|    Austria|        1211|
|   Scotland|        1197|
+-----------+------------+



In [15]:
spark.sql("""
SELECT
    (YEAR(date) / 10) * 10 AS decennie,
    AVG(home_score + away_score) AS moyenne_buts
FROM matches
GROUP BY (YEAR(date) / 10) * 10
ORDER BY decennie
""").show()

+--------+-----------------+
|decennie|     moyenne_buts|
+--------+-----------------+
|  1872.0|              0.0|
|  1873.0|              6.0|
|  1874.0|              3.0|
|  1875.0|              4.0|
|  1876.0|              3.5|
|  1877.0|              3.0|
|  1878.0|              9.0|
|  1879.0|              5.0|
|  1880.0|6.666666666666667|
|  1881.0|4.666666666666667|
|  1882.0|              8.0|
|  1883.0|              4.4|
|  1884.0|              5.0|
|  1885.0|5.428571428571429|
|  1886.0|5.285714285714286|
|  1887.0|4.666666666666667|
|  1888.0|7.142857142857143|
|  1889.0|4.666666666666667|
|  1890.0|              5.5|
|  1891.0|5.666666666666667|
+--------+-----------------+
only showing top 20 rows


In [17]:
spark.sql("""
SELECT tournament, YEAR(date) AS year, COUNT(*) AS nb_matchs
FROM matches
GROUP BY tournament, YEAR(date)
ORDER BY tournament, YEAR(date)
""").show()

+--------------------+----+---------+
|          tournament|year|nb_matchs|
+--------------------+----+---------+
|       AFC Asian Cup|1956|        6|
|       AFC Asian Cup|1960|        6|
|       AFC Asian Cup|1964|        6|
|       AFC Asian Cup|1968|       10|
|       AFC Asian Cup|1972|       13|
|       AFC Asian Cup|1976|       10|
|       AFC Asian Cup|1980|       24|
|       AFC Asian Cup|1984|       24|
|       AFC Asian Cup|1988|       24|
|       AFC Asian Cup|1992|       16|
|       AFC Asian Cup|1996|       26|
|       AFC Asian Cup|2000|       26|
|       AFC Asian Cup|2004|       32|
|AFC Asian Cup qua...|1956|        7|
|AFC Asian Cup qua...|1959|       11|
|AFC Asian Cup qua...|1963|        6|
|AFC Asian Cup qua...|1967|       14|
|AFC Asian Cup qua...|1971|       29|
|AFC Asian Cup qua...|1975|       31|
|AFC Asian Cup qua...|1978|        5|
+--------------------+----+---------+
only showing top 20 rows


In [18]:
spark.sql("""
SELECT home_team, COUNT(*) AS victoires_domicile
FROM matches
WHERE home_score > away_score
GROUP BY home_team
ORDER BY victoires_domicile DESC
""").show()


+-----------+------------------+
|  home_team|victoires_domicile|
+-----------+------------------+
|     Brazil|               331|
|  Argentina|               293|
|    England|               245|
|    Germany|               244|
|     Sweden|               234|
|      Italy|               224|
|    Hungary|               224|
|South Korea|               213|
|     Mexico|               209|
|     France|               205|
|   Scotland|               184|
|    Denmark|               179|
|Netherlands|               177|
|    Austria|               176|
|      Spain|               168|
|    Belgium|               162|
|      Egypt|               160|
|    Uruguay|               156|
|      Chile|               154|
|     Poland|               152|
+-----------+------------------+
only showing top 20 rows


In [19]:
spark.sql("""
SELECT team,
       SUM(win) AS victoires,
       SUM(draw) AS nuls,
       SUM(loss) AS defaites
FROM (
    SELECT home_team AS team,
           CASE WHEN home_score > away_score THEN 1 ELSE 0 END AS win,
           CASE WHEN home_score = away_score THEN 1 ELSE 0 END AS draw,
           CASE WHEN home_score < away_score THEN 1 ELSE 0 END AS loss
    FROM matches
    UNION ALL
    SELECT away_team AS team,
           CASE WHEN away_score > home_score THEN 1 ELSE 0 END,
           CASE WHEN away_score = home_score THEN 1 ELSE 0 END,
           CASE WHEN away_score < home_score THEN 1 ELSE 0 END
    FROM matches
)
GROUP BY team
""").show()


+-----------+---------+----+--------+
|       team|victoires|nuls|defaites|
+-----------+---------+----+--------+
|     Kernow|        5|   1|       3|
|       Chad|       13|  16|      28|
|     Russia|      281| 144|     117|
|   Paraguay|      206| 138|     210|
|   Anguilla|        2|   2|      25|
|      Yemen|       20|  18|      65|
|    Senegal|      166| 107|     126|
|     Sweden|      412| 184|     238|
|      Frøya|        4|   1|       9|
|   Kiribati|        0|   1|       6|
|   Ynys Môn|       18|  10|       8|
|     Guyana|       44|  38|      92|
|Philippines|       10|   4|      80|
|     Jersey|       31|   7|      13|
|    Eritrea|        5|  10|      26|
|   Djibouti|        1|   3|      34|
|      Tonga|       12|   4|      20|
|  Singapore|      114|  69|     206|
|   Malaysia|      196| 121|     184|
|       Fiji|       74|  24|      65|
+-----------+---------+----+--------+
only showing top 20 rows


In [20]:
spark.sql("""
SELECT neutral,
       AVG(home_score + away_score) AS score_moyen
FROM matches
GROUP BY neutral
""").show()


+-------+-----------------+
|neutral|      score_moyen|
+-------+-----------------+
|   true|3.174299065420561|
|  false|3.009568505946143|
+-------+-----------------+



In [21]:
spark.sql("""
SELECT team,
       SUM(marque) - SUM(encaisse) AS goal_average
FROM (
    SELECT home_team AS team,
           home_score AS marque,
           away_score AS encaisse
    FROM matches
    UNION ALL
    SELECT away_team,
           away_score,
           home_score
    FROM matches
)
GROUP BY team
ORDER BY goal_average DESC
""").show()


+-----------+------------+
|       team|goal_average|
+-----------+------------+
|    England|        1008|
|     Brazil|         973|
|    Germany|         753|
|  Argentina|         627|
|South Korea|         623|
|     Sweden|         618|
|    Hungary|         499|
|      Spain|         476|
|      Italy|         467|
|Netherlands|         451|
|     Russia|         419|
|     Mexico|         418|
|   Scotland|         380|
|  Australia|         358|
|   China PR|         341|
|     Zambia|         337|
|       Iraq|         320|
|       Iran|         319|
|      Ghana|         264|
| Costa Rica|         257|
+-----------+------------+
only showing top 20 rows


In [23]:
spark.sql("""
SELECT year, team, victoires,
       ROW_NUMBER() OVER (PARTITION BY year ORDER BY victoires DESC) AS rang
FROM (
    SELECT YEAR(date) AS year, home_team AS team, COUNT(*) AS victoires
    FROM matches
    WHERE home_score > away_score
    GROUP BY YEAR(date), home_team
) AS subquery_wins
""").show()

+----+----------------+---------+----+
|year|            team|victoires|rang|
+----+----------------+---------+----+
|1873|         England|        1|   1|
|1874|        Scotland|        1|   1|
|1876|        Scotland|        2|   1|
|1878|        Scotland|        2|   1|
|1879|         England|        2|   1|
|1880|        Scotland|        2|   1|
|1882|        Scotland|        2|   1|
|1882|           Wales|        2|   2|
|1883|         England|        2|   1|
|1884|        Scotland|        2|   1|
|1884|           Wales|        1|   2|
|1885|        Scotland|        1|   1|
|1885|         England|        1|   2|
|1886|        Scotland|        1|   1|
|1886|           Wales|        1|   2|
|1886|   United States|        1|   3|
|1887|         England|        2|   1|
|1887|        Scotland|        1|   2|
|1887|Northern Ireland|        1|   3|
|1888|        Scotland|        2|   1|
+----+----------------+---------+----+
only showing top 20 rows


In [24]:
spark.sql("""
SELECT tournament, team, victoires
FROM (
    SELECT tournament, team, victoires,
           ROW_NUMBER() OVER (PARTITION BY tournament ORDER BY victoires DESC) AS rang
    FROM (
        SELECT tournament, home_team AS team, COUNT(*) AS victoires
        FROM matches
        WHERE home_score > away_score
        GROUP BY tournament, home_team
    )
)
WHERE rang = 1
""").show()


+--------------------+--------------------+---------+
|          tournament|                team|victoires|
+--------------------+--------------------+---------+
|       AFC Asian Cup|                Iran|       19|
|AFC Asian Cup qua...|            Thailand|       20|
|    AFF Championship|           Singapore|       12|
|African Cup of Na...|               Egypt|       27|
|African Cup of Na...|         Ivory Coast|       32|
|    Afro-Asian Games|               India|        2|
|  Amílcar Cabral Cup|                Mali|       21|
|            Arab Cup|        Saudi Arabia|        9|
|Arab Cup qualific...|             Lebanon|        1|
|        Atlantic Cup|           Argentina|        2|
|          Balkan Cup|            Bulgaria|       16|
|          Baltic Cup|              Latvia|       15|
|Beijing Internati...|            China PR|        4|
|    Bolivarian Games|             Bolivia|        2|
|Brazil Independen...|           Argentina|        3|
|British Championship|      

In [25]:
spark.sql("""
SELECT team,
       SUM(home_win) AS victoires_domicile,
       SUM(away_win) AS victoires_exterieur
FROM (
    SELECT home_team AS team,
           CASE WHEN home_score > away_score THEN 1 ELSE 0 END AS home_win,
           0 AS away_win
    FROM matches
    UNION ALL
    SELECT away_team,
           0,
           CASE WHEN away_score > home_score THEN 1 ELSE 0 END
    FROM matches
)
GROUP BY team
""").show()


+-----------+------------------+-------------------+
|       team|victoires_domicile|victoires_exterieur|
+-----------+------------------+-------------------+
|     Kernow|                 2|                  3|
|       Chad|                10|                  3|
|     Russia|               136|                145|
|   Paraguay|               104|                102|
|   Anguilla|                 1|                  1|
|      Yemen|                12|                  8|
|    Senegal|                93|                 73|
|     Sweden|               234|                178|
|      Frøya|                 3|                  1|
|   Kiribati|                 0|                  0|
|   Ynys Môn|                10|                  8|
|     Guyana|                30|                 14|
|Philippines|                 8|                  2|
|     Jersey|                23|                  8|
|    Eritrea|                 4|                  1|
|   Djibouti|                 1|              